In [1]:
import os
import json
import pandas as pd

conference_name = ['CIKM', 'EMNLP', 'AAAI', 'ICML', 'NeurlPS', 'ICASSP', 'CVPR', 'ICDM']
conference_id = ['1194094125', '1192655580', '1184914352', '1180662882',
                 '1127325140', '1121227772', '1158167855', '1183478919']

In [2]:
conf = pd.read_csv("F:\data\mag\data\preprocessed\conference_ids.csv")
conf.head()

,ConferenceSeriesId,NormalizedName,CitationCount
0,1183478919,ICDM,134343
1,1184914352,AAAI,446128
2,2756032919,ICIP,621
3,1121227772,ICASSP,638525
4,1192655580,EMNLP,315511


In [3]:
conf['ConferenceSeriesId'] = conf['ConferenceSeriesId'].apply(lambda x: str(x))

In [4]:
papers = pd.read_csv("F:\data\mag\data\preprocessed\paperurls.csv")
papers.head()

,PaperId,PaperTitle,YEAR,Date,ConferenceSeriesId,PaperId.1,Urls
0,104302408,maximum margin multiclass nearest neighbors,2014,2014-06-21,1180662882,104302408,https://dblp.uni-trier.de/db/conf/icml/icml201...
1,104302408,maximum margin multiclass nearest neighbors,2014,2014-06-21,1180662882,104302408,http://proceedings.mlr.press/v32/kontorovichb1...
2,104302408,maximum margin multiclass nearest neighbors,2014,2014-06-21,1180662882,104302408,http://proceedings.mlr.press/v32/kontorovichb1...
3,104302408,maximum margin multiclass nearest neighbors,2014,2014-06-21,1180662882,104302408,https://arxiv.org/abs/1401.7898
4,104302408,maximum margin multiclass nearest neighbors,2014,2014-06-21,1180662882,104302408,https://jmlr.org/proceedings/papers/v32/kontor...


In [5]:
papers = papers[['PaperId','YEAR']].drop_duplicates()

In [6]:
papers['PaperId'] = papers['PaperId'].apply(lambda x: str(x))

In [7]:
papers.shape

(75307, 2)

In [8]:
papers[papers['PaperId']=='104302408'].PaperId.item()

'104302408'

In [9]:
#conferenceid, paperid, year, citecount

In [10]:
cid = '1194094125'
cdir = 'F:\data\mag\data\openalex\\' + cid + '_paperID'
file_list = os.listdir(cdir)
file_list[0][:-5]

'127645672'

In [11]:
for i in os.listdir('F:\data\mag\data\paperreferences'):
  t = pd.read_csv(os.path.join('F:\data\mag\data\paperreferences\\', i))
  y = list(t['PaperReferenceId'])
  x = list(t['PaperId'])
  tmp = set(x+y)

In [12]:

for cid in conference_id:
    cdir = 'F:\data\mag\data\openalex\\' + cid + '_paperID'
    file_list = os.listdir(cdir)
    cids = []
    pids = []
    years = []
    ccs = []
    cnm = conf[conf['ConferenceSeriesId']==cid].NormalizedName.item()
    tcsv = 'F:\data\mag\data\paperreferences\\{}_refs.csv'.format(cnm)
    t = pd.read_csv(tcsv)
    print(t.shape)
    y = list(t['PaperReferenceId'])
    x = list(t['PaperId'])
    tmp = set(x+y)
    for file in file_list:
        f = open(os.path.join(cdir, file))
        paperid = file[:-5]
        if int(paperid) in tmp: 
            yy = papers[papers['PaperId']==paperid].YEAR.item()
            try:
                data = json.load(f)
                cids.append(cid)
                pids.append(paperid)
                years.append(yy)
                ccs.append(data['cited_by_count'])
            except:
                pass
    tempdict = {'ConferenceId':cids,'PaperId':pids,'Year':years,'CitationCount':ccs}
    tempdict = pd.DataFrame(tempdict)
    print(cnm," : ", str(len(tempdict)))
    csvnm = "F:\data\mag\data\labels\{}_labeltemp.csv".format(cid)
    tempdict.to_csv(csvnm, encoding='utf-8',index=False)

(2204, 4)
CIKM  :  1964
(15389, 4)
EMNLP  :  4894
(4023, 4)
AAAI  :  3720
(2937, 4)
ICML  :  2121
(4208, 4)
NeurIPS  :  2947
(16339, 4)
ICASSP  :  9336
(68474, 4)
CVPR  :  9782
(1089, 4)
ICDM  :  1240


In [20]:
df[df['Year']==2012].sort_values(by = 'rownum')

,ConferenceId,PaperId,Year,CitationCount,rownum,rowmax,label
1410,1184914352,2406945108,2012,330,1.0,298.0,1
1425,1184914352,2408538552,2012,152,2.0,298.0,1
1311,1184914352,2398690976,2012,142,3.0,298.0,1
62,1184914352,141062567,2012,133,4.0,298.0,1
227,1184914352,168840469,2012,127,5.0,298.0,1
...,...,...,...,...,...,...,...
1114,1184914352,2287866374,2012,0,294.0,298.0,0
1115,1184914352,2287921404,2012,0,295.0,298.0,0
1148,1184914352,2293830845,2012,0,296.0,298.0,0
2074,1184914352,2614616909,2012,0,297.0,298.0,0


In [22]:
for cid in conference_id :
    csvnm = "F:\data\mag\data\labels\{}_labeltemp.csv".format(cid)
    df = pd.read_csv(csvnm)
    df['rownum'] = df.groupby(['Year'])['CitationCount'].rank(method ='first', ascending = False)
    temp  = df.groupby(['Year']).max()[['rownum']].reset_index()
    temp.columns = ['Year','rowmax']
    df = pd.merge(df, temp, how = 'left', on = 'Year')
    df['label'] = df.apply(lambda x: 1 if x['rowmax']*0.1 >= x['rownum'] else 0, axis=1)
    df = df[['PaperId','label']]
    fincsvnm = "F:\data\mag\data\labels\{}_withlabel.csv".format(cid)
    df.to_csv(fincsvnm, encoding ='utf-8', index=False)